<a href="https://colab.research.google.com/github/KLAU-S/SRIP2022/blob/main/Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
#from keras.datasets import mnist #importing from keras is not working since we require only two dataframes train and test
#from torchvision.datasets import MNIST #imports literal images (don't need)
#from torch.utils.data import DataLoader #DataLoader loads the mnist img data that we will pass in the MLP

In [2]:
import jax.numpy as jnp
from jax.scipy.special import logsumexp
import jax
from jax import jit, vmap, pmap, grad, value_and_grad

The code below imports dataset which has images as elements if imported from torchvision, but need numpy array type elements

In [3]:
# train_dataset = MNIST(root='train_mnist', train=True, download=True, transform=None)
# test_dataset = MNIST(root='test_mnist', train=False, download=True, transform=None

**The MNIST data is already split between 60,000 28 x 28 pixel training images and 10,000 28 x 28 pixel image if imported from keras**

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [5]:
print(x_train[0].shape)
print(y_train[0].shape)
print(x_test[0].shape)
print(y_test[0].shape)

(28, 28)
()
(28, 28)
()


In [6]:
y_train[0].shape

()

In [7]:
x_test[0].shape

(28, 28)

In [8]:
y_test[0].shape

()

Since its already splitted into 60000 and 10000 we can concatanate and then split into 80:20 

In [9]:
x = np.concatenate([x_train, x_test])
y = np.concatenate([y_train, y_test])

In [10]:
for i in range (10):
  print(y[i])

5
0
4
1
9
2
1
3
1
4


In [11]:
print(type(x_train))

<class 'numpy.ndarray'>


### Splitting MNIST dataset with 80:20 train:test

In [12]:
#training_data, testing_data = train_test_split(x, y, test_size=0.2, random_state=42)
train_ratio = 0.8
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=(1-train_ratio), random_state=42)

#validating
###since, 70000 * 80% = 56000 splitting success

In [13]:
print(X_train.shape) 
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(56000, 28, 28)
(56000,)
(14000, 28, 28)
(14000,)


In [14]:
#concat_train = pd.concat([X_train, pd.DataFrame(Y_train)])

In [15]:
img = X_train[0]
img2 = Y_train[0]
img.shape

(28, 28)

In [16]:
img2.shape

()

In [17]:
np.ravel(X_train)
img.shape

(28, 28)

#ALL ACTIVATION FUCTIONS

In [18]:
def relu(x):
    return np.maximum(x, np.zeros_like(x))

In [19]:
def softmax(x):
    x = x-np.max(x)
    return np.exp(x) / np.sum(np.exp(x))

In [20]:
def crossentropy(y, yhat):
    return -np.nansum(y*np.log(yhat), axis=1)

##Gradient Descent 
- It is a first-order optimization technique used to find the local minimum or optimize the loss function. It is also known as the parameter optimization technique.
- It is easy to find the value of slope and intercept using a **closed-form solution** But when you work in Multidimensional data then the technique is so costly and takes a lot of time Thus it fails here. So, the new technique came as Gradient Descent which finds the minimum very fastly.

- An equation is said to be a **closed-form solution** if it solves a given problem in terms of functions and mathematical operations from a given generally-accepted set. For example, an infinite sum would generally not be considered closed-form.

##To implement gradient descent we can use jax.grad function but implementing from scratch<br>
```python
jax.grad(fun, argnums=0, has_aux=False, holomorphic=False, allow_int=False, reduce_axes=())[source]
```



In [21]:
def gradient_descent(self, x, y, iterations):
        for i in range(iterations):
            Xi = x
            Xj = self.sigmoid(Xi, self.wij)
            yhat = self.sigmoid(Xj, self.wjk)
            # gradients for hidden to output weights
            g_wjk = np.dot(Xj.T, (y - yhat) * self.sigmoid_derivative(Xj, self.wjk))
            # gradients for input to hidden weights
            g_wij = np.dot(Xi.T, np.dot((y - yhat) * self.sigmoid_derivative(Xj, self.wjk), self.wjk.T) * self.sigmoid_derivative(Xi, self.wij))
            # update weights
            self.wij += g_wij
            self.wjk += g_wjk
        print('The final prediction from neural network are: ')
        print(yhat)